In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn, Tensor

In [2]:
from transformer import TransformerModel
from transformer_config import load_transformer_config
from preprocessor import Preprocessor, numpy_to_pytorch, load_preprocessor_config


In [3]:
transformer_config = load_transformer_config("./configs/transformer/default.yaml")
model = TransformerModel(transformer_config)
transformer_config.dict()


{'seq_length': 10,
 'n_classes': 3288,
 'training_data_path': './data/ratings-split0.csv',
 'validation_data_path': './data/ratings-split0.csv',
 'model_spec': {'d_model': 50, 'nhead': 2, 'd_hid': 50, 'nlayers': 2},
 'training_spec': {'device': 'cpu',
  'epochs': 1,
  'iter_save': 10,
  'batch_size': 50,
  'lr': 0.001,
  'dropout': 0.3,
  'criterion': 'CrossEntropyLoss',
  'optimizer': {'name': 'Adam'},
  'scheduler': {'name': 'StepLR', 'step_size': 1.0, 'gamma': 0.99}}}

In [4]:
preprocessor_config = load_preprocessor_config("./configs/preprocessor/default.yaml")
preprocessor = Preprocessor(**preprocessor_config.dict())

In [5]:
X_train, y_train = numpy_to_pytorch(preprocessor.splits[0], preprocessor_config.seq_length, transformer_config.training_spec.device)
X_valid, y_valid = numpy_to_pytorch(preprocessor.splits[1], preprocessor_config.seq_length, transformer_config.training_spec.device)
X_test, y_test = numpy_to_pytorch(preprocessor.splits[2], preprocessor_config.seq_length, transformer_config.training_spec.device)

In [6]:
model.train_model(X_train, y_train, X_valid, y_valid)


/usr/local/lib/python3.8/dist-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


-----------------------------------------------------------------------------------------
| end of epoch   1 | time:  5.34s | valid loss  1.81 | valid ppl     6.12
-----------------------------------------------------------------------------------------


/home/leon/projects/sequifier/transformer.py:223: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  x = x + self.pe[:x.size(0)]


In [7]:
from inferer_config import load_inferer_config
inferer_config = load_inferer_config("./configs/inferer/default.yaml")

In [10]:
from inferer import Inferer

inferer = Inferer("models/sequifier-4eecb3ee-best.onnx")
x_test = X_test.detach().cpu().numpy()
probs = inferer.infer_probs(x_test.T)
probs

array([[-5.4729342 ,  1.299694  ,  0.54408395, ..., -1.3184454 ,
        -2.5099525 , -3.4055407 ],
       [-4.854726  ,  0.76395094,  0.0486614 , ..., -1.1098588 ,
        -2.9533162 , -3.3575296 ],
       [-5.4430447 ,  0.95491594,  0.22478807, ..., -1.398021  ,
        -2.8561687 , -4.1762753 ],
       ...,
       [-5.227994  ,  0.68309355,  0.1911543 , ..., -1.5813189 ,
        -2.539854  , -4.082448  ],
       [-5.689005  ,  0.9216385 ,  0.34630784, ..., -1.9468362 ,
        -2.665267  , -3.8618102 ],
       [-5.4838233 ,  0.8872019 , -0.13988963, ..., -1.429875  ,
        -2.951868  , -3.9098978 ]], dtype=float32)